In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *
import random
import numpy as np
import time

#### Datasets: Import + Preprocessing


In [ ]:
# Instance1c = Instance 1 + Added nodes in frontiers reprenseting water sources

# Read
df = pd.read_excel('/datasets/instance_1c.xlsx','RegionLevel')
matrix = df.to_numpy() #matrice
var2 = pd.read_excel('/datasets/instance_1c.xlsx',sheet_name=0)
slr = var2['Value'].to_numpy()[0] #sea elevation
var3 = pd.read_excel('/datasets/instance_1c.xlsx',sheet_name=1)
asset0 = var3.to_numpy()
assets = [(asset0[i][1]+1,asset0[i][2]+1) for i in range(len(asset0))]
start_nodes = []



#Add borders
if matrix[1,1] < slr: 
    matrix[0,0] = 2
if matrix[len(matrix)-2,len(matrix)-2] < slr:
    matrix[len(matrix)-1,len(matrix)-1] = 2
if matrix[1,len(matrix)-2] < slr:
    matrix[0,len(matrix)-1] = 2
if matrix[len(matrix)-2,1] < slr:
    matrix[len(matrix)-1,0] = 2
for j in range(1,len(matrix)-1):
    if matrix[1,j] < slr and j != 1 and j!= len(matrix)-2 : 
        #matrix[0,j-1] = 2
        matrix[0,j] = 999
        #matrix[0,j+1] = 2

    if matrix[len(matrix)-2,j] < slr and j!=1:
        #matrix[len(matrix)-1,j-1] = 2
        matrix[len(matrix)-1,j] = 2
        #matrix[len(matrix)-1,j+1] = 2
for i in range(1,len(matrix)-1):
    if matrix[i,1] < slr and i!=len(matrix)-2 and i!=1:
        #matrix[i-1,0] = 2
        matrix[i,0] = 2
        #matrix[i+1,0] = 2
    if matrix[i,len(matrix)-2] < slr and i !=1 and i != len(matrix)-2:
        #matrix[i-1,len(matrix)-1] = 2
        matrix[i,len(matrix)-1] = 2
        #matrix[i+1,len(matrix)-1] = 2

#### Graph generation (NetworkX)

In [ ]:
def generate_graph(matrix, slr):
    G = nx.Graph()
    # Add nodes to the graph
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] < slr :
                G.add_node((i,j))

    for i, j in G.nodes():
        if  i == 0 or j == 0 or i == len(matrix)-1 or j == len(matrix)-1:
            if i ==0 and j==0:
                G.add_edge((i,j),(i+1,j+1),cost=slr-matrix[i+1,i+1])
            elif i==len(matrix)-1 and j == len(matrix)-1:
                G.add_edge((i,j),(i-1,j-1),cost=slr-matrix[i-1,j-1])
            elif i ==0 and j == len(matrix)-1 : 
                G.add_edge((i,j),(i+1,j-1),cost=slr-matrix[i+1,j-1])
            elif i==len(matrix)-1 and j == 0:
                G.add_edge((i,j),(i-1,j+1),cost=slr-matrix[i-1,j+1])
            elif i==0:
                G.add_edge((i,j),(i+1,j),cost=slr-matrix[i+1,j])
            elif i==len(matrix)-1:
                G.add_edge((i,j),(i-1,j),cost=slr-matrix[i-1,j])
            elif j==0:
                G.add_edge((i,j),(i,j+1),cost=slr-matrix[i,j+1])
            elif j==len(matrix)-1:
                G.add_edge((i,j),(i,j-1),cost=slr-matrix[i,j-1])

            
        else:
            if i+1 < matrix.shape[0]-1 and matrix[i+1, j] < slr:
                G.add_edge((i, j), (i+1, j),cost=slr-min(matrix[i,j],matrix[i+1,j]))
            if j+1 < matrix.shape[1]-1 and matrix[i, j+1] < slr :
                G.add_edge((i, j), (i, j+1),cost=slr-min(matrix[i,j],matrix[i,j+1]))
            if i-1 >= 0 and matrix[i-1, j] < slr:
                G.add_edge((i, j), (i-1, j),cost=slr-min(matrix[i,j],matrix[i-1,j]))
            if j-1 >= 0 and matrix[i, j-1] < slr:
                G.add_edge((i, j), (i, j-1),cost=slr-min(matrix[i,j],matrix[i,j-1]))
            if i+1 < matrix.shape[0]-1 and j+1 < matrix.shape[1]-1 and matrix[i+1, j+1] < slr:
                G.add_edge((i, j), (i+1, j+1),cost=slr-min(matrix[i,j],matrix[i+1,j+1]))
            if i-1 >= 0 and j+1 < matrix.shape[1]-1 and matrix[i-1, j+1] < slr:
                G.add_edge((i, j), (i-1, j+1),cost=slr-min(matrix[i,j],matrix[i-1,j+1]))
            if i+1 < matrix.shape[0]-1 and j-1 >= 1 and matrix[i+1, j-1] < slr:
                G.add_edge((i, j), (i+1, j-1),cost=slr-min(matrix[i,j],matrix[i+1,j-1]))
            if i-1 >= 1 and j-1 >= 1 and matrix[i-1, j-1] < slr:
                G.add_edge((i, j), (i-1, j-1),cost=slr-min(matrix[i,j],matrix[i-1,j-1]))


    return G

#Graph:
G = generate_graph(matrix, slr)

#### Genetic Algorithm

In [ ]:
#Genetic Algorithm:
num_vars = len(G.edges.data())-1
Cost = []
D = []
L = list(G.edges.data())
for i in range(len(L)):
    H = set([L[i][0],L[i][1]])
    D.append(H)
    Cost.append(L[i][2]['cost'])

#Population initiale de taille N
routes=[]
start_nodes = []
K=[]

for node in G.nodes():
    if node[0] == 0 or node[0] == matrix.shape[0]-1 or node[1] == 0 or node[1] == matrix.shape[1]-1:
        start_nodes.append(node)
# Trouver les routes
for start_node in start_nodes:
    for asset in assets:
        #for path in nx.all_simple_paths(G, source=start_node, target=end_node):
        if nx.has_path(G,start_node,asset):
            for path in nx.all_shortest_paths(G, source=start_node, target=asset):
                routes.append(path)
for route in routes:
    for i in range(len(route)-1):
        K.append(D.index(set([route[i],route[i+1]])))



def check_feasible(x):
    Gc = G.copy()
    for i in range(len(x)):
        R=list(D[i])
        if x[i] == 1: 
            k = R[0]
            l = R[1]
            Gc.remove_edge(k,l)
    
    for node in start_nodes:
        for asset in assets:
            if nx.has_path(Gc,node,asset)==True and node != asset : 
                return(False)
    return(True)

def population_initiale(N):
    X=[]
    for i in range(N):
        x = [0]*len(G.edges())
        for m in K :
            x[m]=random.choices([0,1], weights=weights, k = 1)[0]
        if check_feasible(x): 
            X.append(x)
        else: 
            while check_feasible(x) == False:
                for m in K:
                    x[m]=random.choices([0,1], weights=weights, k = 1)[0]
            X.append(x)
    return(X)

def fitness(X):
    D={}
    for x in X:
        s=0
        for i in range(len(x)):
            if x[i]==1: s+=Cost[i]
        D[s]=x
    Costmin = min(D.keys())
    xmin=D[Costmin]
    return(xmin,Costmin)

def parents(pop,l):
    X = random.choices(pop, k = l)
    parent1 = fitness(X)[0]
    X.remove(parent1)
    parent2 = fitness(X)[0]
    return parent1,parent2

def crossover(parent1,parent2):
    pc = random.randint(0,len(parent1)-1)
    child1 = parent1[:pc+1] + parent2[pc+1:]
    child2 = parent2[:pc+1] + parent1[pc+1:]
    return child1,child2

def mutation(child1,child2, mutationProbability):
        #child1:
    mutationPlace = np.random.randint(0,len(child1)-1)

    if child1[mutationPlace] == 0 : 
        child1[mutationPlace] = 1
    else : 
        child1[mutationPlace] = 0
      
        #child2:
    mutationPlace = np.random.randint(0,len(child2)-1)

    if child2[mutationPlace] == 0 : 
        child2[mutationPlace] = 1
    else : 
        child2[mutationPlace] = 0

    
    while check_feasible(child1)== False:
        mutationPlace = np.random.randint(0,len(child1)-1)
    
        if child1[mutationPlace] == 0 : 
            child1[mutationPlace] = 1
        else : 
            child1[mutationPlace] = 0
    while check_feasible(child2)== False:
        mutationPlace = np.random.randint(0,len(child2)-1)

        if child2[mutationPlace] == 0 : 
            child2[mutationPlace] = 1
        else : 
            child2[mutationPlace] = 0

    return child1,child2



#### Grid Search over nbgen(# of generations) and N

In [6]:
weights = [0.9, 0.1]
mutationProbability = 0.5


#on varie N et nbgen:
S = []
for nbgen in range(5,1000,1):
    for N in range(3,100,1):
        start_time = time.time()
        population = population_initiale(N)
        Y=[]
        k = 3
        for gen in range(nbgen):
            X=[]
            while len(X)<N:
                parent1,parent2 = parents(population,k)
                child1,child2 = crossover(parent1,parent2)
                child1,child2 = mutation(child1,child2, mutationProbability)
                X.append(child1)
                X.append(child2)
            Y.append(fitness(X)[0])
            population = X.copy()
        x,c = fitness(Y)
        end_time = time.time()
        execution_time = end_time - start_time
        S.append([c,N,nbgen,execution_time])
        print([c,N,nbgen,execution_time])

# Output: Optimal cost,N,nbgen,execution_time

